In [1]:
import time
import numpy as np
import matplotlib.pylab as plt

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from tensorflow.keras import layers

In [3]:
# Part 1: Load the Cats vs. Dogs Dataset
# We will use TensorFlow Datasets to load the Dogs vs Cats dataset.

# -------------------------------------------------------------------

# If you look at the TensorFlow Datasets documentation you will see that the name of 
# the cats & dogs dataset is cats_vs_dogs. You can also see that this dataset is only split 
# into a TRAINING set. You will therefore have to use tfds.splits to split 
# this training set into to a training_set and a validation_set.
# Do a [80, 20] split such that 80 corresponds to the training_set
# and 20 to the validation_set. Then load the cats_vs_dogs dataset using tfds.load.
# Make sure the tfds.load function uses the parameters you need,
# and also make sure it returns the dataset info, so we can retrieve information
# about the datasets.

In [4]:
splits=tfds.Split.ALL.subsplit(weighted=(80,20))
splits,info=tfds.load('cats_vs_dogs',with_info=True,as_supervised=True,split=splits)
(train_examples,validation_examples)=splits

In [5]:
# The images in the Dogs vs. Cats dataset are not all the same size.
# So, we need to reformat all images to the resolution expected by MobileNet (224, 224)

In [6]:
def format_image(image, label):
  # `hub` image modules exepct their data normalized to the [0,1] range.
  image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
  return  image, label

num_examples = info.splits['train'].num_examples

BATCH_SIZE = 32
IMAGE_RES = 224

train_batches      = train_examples.cache().shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.cache().map(format_image).batch(BATCH_SIZE).prefetch(1)

In [7]:
# Part 2: Transfer Learning with TensorFlow Hub
# We will now use TensorFlow Hub to do Transfer Learning.

# Create feature_extractor using MobileNet v2. Remember that the partial model from TensorFlow Hub
# (without the final classification layer) is called a feature vector.
# Go to the TensorFlow Hub documentation to see a list of available feature vectors.
# Click on the tf2-preview/mobilenet_v2/feature_vector.
# Read the documentation and get the corresponding URL to get the MobileNet v2 feature vector.
# Finally, create a feature_extractor by using hub.KerasLayer with the correct
# input_shape parameter.

In [8]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor = hub.KerasLayer(URL,
                                   input_shape=(IMAGE_RES, IMAGE_RES,3))

In [9]:
# Freeze the variables in the feature extractor layer, so that the training
# only modifies the final classifier layer.

In [10]:
feature_extractor.trainable = False

In [11]:
# Attach a classification head
# Now wrap the hub layer in a tf.keras.Sequential model, and add a new classification layer.

# Create a tf.keras.Sequential model, and add the pre-trained model and the new
# classification layer. Remember that the classification layer must have the same number
# of classes as our Cats&Dogs dataset. Finally print a summary of the Sequential model.

In [12]:
model=tf.keras.Sequential([feature_extractor,
                          layers.Dense(2,activation='softmax'),
                          ])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [13]:
# Train the model
# In the cell bellow train this model like any other,
# by first calling compile and then followed by fit.
# Make sure you use the proper parameters when applying both methods.
# Train the model for only 2 epochs.

# We now train this model like any other, by first calling compile followed by fit.

In [15]:
model.compile(
  optimizer='adam', 
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

EPOCHS = 1
history = model.fit(train_batches,
                    epochs=EPOCHS,
                    validation_data=validation_batches)

582/582 [==============================] - 3761s 6s/step - loss: 0.3246 - accuracy: 0.9895


InternalError: Cache should only be read after it has been completed. [Op:MakeIterator]